# Ćwiczenie 4 - Regresja i klasyfikacja
## Implementacja drzew decyzyjnych tworzonych algorytmem ID3 z ograniczeniem maksymalnej głębokości drzewa.
Ćwiczenie wykonała Małgorzata Kozłowska, 3186810

### Importowanie modułów
- potrzebnych do wykonania ćwiczenia. W realizacji algorytmu oraz jego badania korzystam z biblioteki typing, numpy, pandas,oraz sklearn.

In [12]:
from typing import List, Dict
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

### Implementacja metod potrzebnych do przetworzenia danych

In [3]:
def extract_data_from_csv_file(file_handle):
    """
    Extracts data from file. Converts it into dataframe pandas type.
    :param file_handle:
    :return: Data represented as pandas DataFrame type.
    """
    dataset = pd.read_csv(file_handle, header=None)
    return dataset


def prepare_data_for_analysis(dataset: pd.DataFrame, column_names: List[str]):
    """
    Adds column names for better code and data analysis.
    :return: Modified data with column names
    """
    dataset.columns = column_names
    return dataset


def divide_dataset_for_target_attribute(dataset: pd.DataFrame, target_attribute: str):
    """
    Divides provided dataset into two:
    1) contains all the attributes (and their values) on which our ID3 algorithm will learn "the pattern" of the data.
    2) contains the target attribute's values, which later will be predicted by the ID3 tree
    :param dataset: provided dataset for analysis
    :param target_attribute: target attribute on account of which the dataset will be analysed
    :return: two datasets (as array and column)
    """
    dataset_to_analyse = dataset.drop([target_attribute], axis=1)
    target_attribute_column = dataset[target_attribute]
    return dataset_to_analyse, target_attribute_column

### Implementacja metod związanych z rachunkami wykorzystywanymi w algorytmie ID3

In [4]:
def information_gain(dataset_entropy: float, attribute_name: str, attribute_average_information: float):
    """
    Method that calculates information gain for the provided attribute of the dataset.
    :param dataset_entropy: calculated entropy for the whole provided (earlier) dataset
    :param attribute_name: analysed attribute
    :param attribute_average_information: calculated earlier average information for the given attribute
    :return: name of the attribute and its information gain
    """
    information_gain = dataset_entropy - attribute_average_information
    return attribute_name, information_gain


def find_best_attribute(attributes_information_gain: Dict[str, float]):
    """
    Method that finds the best attribute based on its information gain.
    :param attributes_information_gain: list of information gains for each attribute in the dataset.
    :return: attribute which provides the highest information gain.
    """
    return max(attributes_information_gain, key=lambda value: attributes_information_gain[value])


def entropy(target_attribute_values: pd.Series):
    """
    Method that calculates entropy of the provided target attribute based on the formula
    :param target_attribute_values:
    :return: calculated entropy for the provided attribute
    """

    class_values_categorized = target_attribute_values.value_counts()
    total_class_number = class_values_categorized.sum()
    entropy = sum([-(distinct_value_counts / total_class_number)*np.log2(distinct_value_counts / total_class_number)
                   for distinct_value_counts in class_values_categorized])
    return entropy


def calculate_information_gains_for_each_attribute(dataset_to_analyse: pd.DataFrame, target_attribute: pd.Series):
    """
    Wrapper function that calculates information gain for each attribute in provided dataset.
    :param dataset_to_analyse:
    :param target_attribute: target attribute in
    :return: dictionary which stores data in the form: {key = attribute's name: value = information gain of that attribute}
    """
    attribute_information_gains = {}
    for each_column in dataset_to_analyse.columns:
        attr_name, information = information_gain(entropy(target_attribute),
                                                  average_information(dataset_to_analyse[each_column], target_attribute)[0],
                                                  average_information(dataset_to_analyse[each_column], target_attribute)[1])
        attribute_information_gains[attr_name] = information
    return attribute_information_gains


def average_information(data_attribute_subset: pd.DataFrame, target_attribute_values: pd.Series):
    """
    Method that calculates average information obtained from the provided attribute in the dataset
    It iterates through each value (of the given attribute) and calculates entropy of the target attribute
    (for that attribute value).
    :param data_attribute_subset: series of the given attribute
    :param target_attribute_values: target attribute values
    :return: attributes name, average information for the given attribute
    """
    values_to_analyse = data_attribute_subset.unique()
    total_class_number = data_attribute_subset.value_counts().sum()
    attribute_average_information = 0

    for each_value in values_to_analyse:
        value_fraction = data_attribute_subset.value_counts()[each_value].sum()/total_class_number
        selected_rows = data_attribute_subset.where(data_attribute_subset == each_value)
        rows_to_analyse = target_attribute_values.copy().loc[selected_rows.dropna().index.values.tolist()]
        attribute_average_information += value_fraction*entropy(rows_to_analyse)
    return data_attribute_subset.name, attribute_average_information

### Implementacja klasy ID3Node


In [5]:
class ID3Node:
    def __init__(self, attribute_name: str, values_of_attribute, children_nodes: ['ID3Node'] = None):
        self._attribute_name = attribute_name
        self._tree_branches_dictionary = {value: child_node for value, child_node in zip(values_of_attribute, children_nodes)}
        self._values_of_attribute = values_of_attribute
        self._children_nodes = children_nodes

    def predict_next_node(self, data_row: pd.Series):
        """
        Method that for each provided row iterates through tree branches dictionary in order to find the value of
        the target attribute. If the provided node does not exist (there is a Key Error) the value chosen is the most
        frequent value from the node's children
        :param data_row: one row of data
        """
        value = data_row[self._attribute_name]
        try:
            next_node = self._tree_branches_dictionary[value]

            if not isinstance(next_node, ID3Node):
                return next_node

        except KeyError:
            most_frequent_target_attribute_value = []
            for child_node in self._tree_branches_dictionary.values():
                if not isinstance(child_node, ID3Node):
                    most_frequent_target_attribute_value.append(child_node)
                    continue
                most_frequent_target_attribute_value.append(child_node.predict_next_node(data_row))

            most_frequent_value = max(set(most_frequent_target_attribute_value), key=most_frequent_target_attribute_value.count)
            return most_frequent_value
        return next_node.predict_next_node(data_row)

### Implementacja klasy ID3Tree


In [6]:
class ID3Tree:
    def __init__(self, max_depth: int):
        self._max_depth = max_depth
        self._root = None

    def get_max_depth(self):
        """
        ID3 tree maximum depth getter.
        :return: maximum depth of the ID3 tree.
        """
        return self._max_depth

    def get_root(self):
        """
        ID3 tree root getter.
        :return: root of the ID3 tree.
        """
        return self._root

    def set_root(self, new_root: ID3Node):
        """
        ID3 tree root setter.
        :param new_root: ID3 Node object, which will be the root of the ID3 tree.
        """
        self._root = new_root

    def build_ID3_tree(self, dataset_to_analyse: pd.DataFrame, target_attribute: pd.Series, depth: int):
        """
        Creates ID3 tree structure by recursively finding best attribute to divide the current set. When the given value
        of the attribute has homogeneous target attribute values or the tree reached its maximum depth (provided by user)
        or there are no columns (attributes) to analyse we return the leaf node
        equal to 0,
        :param dataset_to_analyse: provided part of the data on which the model will learn dependencies between data
        :param target_attribute:
        :param depth:
        :return: an ID3 tree with newly created ID3 Nodes
        """
        total_class_number = target_attribute.value_counts()
        if len(total_class_number) == 1 or depth == 0 or len(dataset_to_analyse.columns) == 0:
            return total_class_number.index.tolist()[0]
        attributes_information_gain = calculate_information_gains_for_each_attribute(dataset_to_analyse, target_attribute)
        division_attribute = find_best_attribute(attributes_information_gain)
        values_to_analyse = dataset_to_analyse[division_attribute].unique()
        children_nodes = [self.build_ID3_tree(dataset_to_analyse[dataset_to_analyse[division_attribute] == value].drop(division_attribute, axis=1),
                                          target_attribute[dataset_to_analyse[division_attribute] == value], depth - 1) for value in values_to_analyse]
        return ID3Node(division_attribute, values_to_analyse, children_nodes)

    def predict_target_attribute_value(self, train_dataset: pd.DataFrame):
        """
        Method that for the provided dataset iterates through nodes (starting from the root node
        - which is the ID3Tree instance attribute) in order to find target attribute value.
        Predict_next_node method (which is a method of the ID3Node class) is called - it iterates
        through each row of the provided dataset.
        :param train_dataset: dataset to analyse
        :return: series of the predicted target attribute values
        """
        root_node = self.get_root()

        return train_dataset.apply(lambda data_row: root_node.predict_next_node(data_row), axis=1)

### Implementacja metody znajdującej optymalną głębokość drzewa algorytmu ID3

In [66]:
def find_differences(tree: ID3Tree, validation_dataset: pd.DataFrame, validation_target: pd.Series):
    predicted_validation_target = tree.predict_target_attribute_value(validation_dataset)
    number_of_different_values_in_rows = validation_target.compare(predicted_validation_target).count()[0]
    return number_of_different_values_in_rows

def find_best_depth(trees_for_analysis: List[ID3Tree]):
    depth_and_differences = {}
    for each_tree_index in range(len(trees_for_analysis)):
        depth_and_differences[each_tree_index + 1] = find_differences(trees_for_analysis[each_tree_index], x_valid, y_valid)
    best_max_depth = min(depth_and_differences, key=depth_and_differences.get)
    return best_max_depth, depth_and_differences

def evaluate_best_depth(depth_and_differences: Dict):
    best_depth, depth_and_wrong_evaluation_number = depth_and_differences
    print("Najbardziej optymalna głębokość drzewa ID3 dla zbioru danych otrzymanych z pliku 'breast_cancer.csv' to: " + str(best_depth))
    print("Liczba źle skategoryzowanych danych: " + str(depth_and_wrong_evaluation_number[best_depth]))

def evaluate_depths(depths_and_differences: Dict):
    print("Dla głębokości drzewa ID3 równej: ")
    for each_key in depths_and_differences:
        print(str(each_key) + " - zostało źle sklasyfikowanych: " +str(depths_and_differences[each_key]) + " danych.")

In [62]:
def create_trees_for_analysis(dataset: pd.DataFrame, target_attribute: pd.Series, depth_range: int):
    list_of_trees = []
    for each_depth in range(1, depth_range + 1):
        new_tree = ID3Tree(each_depth)
        new_tree.set_root(new_tree.build_ID3_tree(dataset, target_attribute, new_tree.get_max_depth()))
        list_of_trees.append(new_tree)
    return list_of_trees

## Badanie klasyfikatorów
Przygotowujemy dane do analizy

In [63]:
data_frame = extract_data_from_csv_file('breast_cancer.csv')
breast_cancer_columns = ["Class", "age", "menopause", "tumor size", "inv nodes", "node caps", "deg malig", "breast", "breast quad", "irradiat"]
formatted_data = prepare_data_for_analysis(data_frame, breast_cancer_columns)
data, target = divide_dataset_for_target_attribute(formatted_data, 'irradiat')

Zbiór danych dzielimy odpowiednio na zbiory: trenujący, walidacyjny oraz testowy.

In [69]:
x_train, x_rem, y_train, y_rem = train_test_split(data, target, train_size=0.8)
x_valid, x_test, y_valid, y_test = train_test_split(x_rem, y_rem, test_size=0.5)

 Znajdujemy najlepszą głębokość drzewa ID3.

In [70]:
trees_for_analysis = create_trees_for_analysis(x_train, y_train, 6)
depth_evaluation = find_best_depth(trees_for_analysis)
best_depth, depths_and_mistakes = depth_evaluation
evaluate_best_depth(depth_evaluation)
evaluate_depths(depths_and_mistakes)

Najbardziej optymalna głębokość drzewa ID3 dla zbioru danych otrzymanych z pliku 'breast_cancer.csv' to: 2
Liczba źle skategoryzowanych danych: 6
Dla głębokości drzewa ID3 równej: 
1 - zostało źle sklasyfikowanych: 9 danych.
2 - zostało źle sklasyfikowanych: 6 danych.
3 - zostało źle sklasyfikowanych: 7 danych.
4 - zostało źle sklasyfikowanych: 9 danych.
5 - zostało źle sklasyfikowanych: 10 danych.
6 - zostało źle sklasyfikowanych: 9 danych.


### Wnioski

Można zauważyć, że w drzewach ID3 pomimo zwiększania maksymalnej głębokości ewaluacji atrybutów precyzja odnajdywania odpowiedniej wartości ze zbioru atrybutów docelowych (target attributes) maleje. Dochodzi do błędnej klasyfikacji danych - mamy do czynienia ze zjawiskiem tzw. przeuczenia drzewa.

Dla danych zadanych w poleceniu najlepsza głębokość to 2.

In [72]:
best_depths = {}
for _ in range(20):
    x_train, x_rem, y_train, y_rem = train_test_split(data, target, train_size=0.8)
    x_valid, x_test, y_valid, y_test = train_test_split(x_rem, y_rem, test_size=0.5)
    trees_for_analysis = create_trees_for_analysis(x_train, y_train, 6)
    best_depth = find_best_depth(trees_for_analysis)[0]
    if best_depth not in best_depths.keys():
        best_depths[best_depth] = 1
    best_depths[best_depth] += 1
print(best_depths)


{2: 12, 1: 8, 3: 3}
